# Assignment 2 | Image Classification, December 2021

- [Artificial Neural Networks and Deep Learning 2021 - Homework 2](https://codalab.lisn.upsaclay.fr/competitions/621)

## Three convolutioneers

- *Aleksandra Krajnovic*
- *Iva Milojkovic*
- *Mariusz Wiśniewski*

### Connect to Drive

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
# %cd /gdrive/My Drive/Colab Notebooks/ANN_Homework2

### Import libraries

In [ ]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

plt.rc('font', size=16)
import warnings

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### Set seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Utility function to create folders and callbacks for training

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime


def create_folders_and_callbacks(model_name):
    exps_dir = os.path.join('experiments')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)

    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]

    # Model checkpoint
    # ----------------
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(ckpt_dir, 'cp'),
        save_weights_only=False,
        save_best_only=False)
    callbacks.append(ckpt_callback)

    # Visualize Learning on Tensorboard
    # ---------------------------------
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)

    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                                 profile_batch=0,
                                                 histogram_freq=1)
    callbacks.append(tb_callback)

    return callbacks

### Exploration Data Analysis (EDA)


In [ ]:
dataset = pd.read_csv('/kaggle/input/training/Training.csv')
print(dataset.shape)
dataset.head()

In [ ]:
dataset.info()

In [ ]:
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17, 17))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()


inspect_dataframe(dataset, dataset.columns)

Normalization

In [ ]:
# Data normalization

X_train_raw = dataset
X_min = X_train_raw.min()
X_max = X_train_raw.max()
X_train_raw = (X_train_raw - X_min) / (X_max - X_min)

print(X_train_raw.shape)
print(dataset)

# Plot dataset after normalization
inspect_dataframe(X_train_raw, X_train_raw.columns)

In [ ]:
window = 1600
stride = 100

In [ ]:
future = dataset[-window:]
future = (future - X_min) / (X_max - X_min)
future = np.expand_dims(future, axis=0)
future.shape

In [ ]:
def build_sequences(df, p_target_labels=dataset.columns, p_window=1600, p_stride=100, p_telescope=864):
    # Sanity check to avoid runtime errors
    assert p_window % p_stride == 0
    ds = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[p_target_labels].copy().values
    padding_len = len(df) % p_window

    if padding_len != 0:
        # Compute padding length
        padding_len = p_window - len(df) % p_window
        padding = np.zeros((padding_len, temp_df.shape[1]), dtype='float32')  # Change to float32
        temp_df = np.concatenate((padding, df))
        padding = np.zeros((padding_len, temp_label.shape[1]), dtype='float32')
        temp_label = np.concatenate((padding, temp_label))
        assert len(temp_df) % p_window == 0

    for idx in np.arange(0, len(temp_df) - p_window - p_telescope, p_stride):
        ds.append(temp_df[idx:idx + p_window])
        labels.append(temp_label[idx + p_window:idx + p_window + p_telescope])

    ds = np.array(ds)
    labels = np.array(labels)
    return ds, labels

### Multivariate Forecasting (Direct)

In [ ]:
target_labels = dataset.columns
telescope = 864

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape

In [ ]:
def inspect_multivariate(X, y, columns, p_telescope, idx=None):
    if idx is None:
        idx = np.random.randint(0, len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17, 17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0, :, i])), X[idx, :, i])
        axs[i].scatter(np.arange(len(X[0, :, i]), len(X_train[0, :, i]) + p_telescope), y[idx, :, i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0, 1)
    plt.show()

In [ ]:
inspect_multivariate(X_train, y_train, target_labels, telescope)

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 200

# Source: https://arxiv.org/abs/1711.00489
# Is it a good tradeoff? Memory vs just decreasing a value

In [ ]:
def build_stacked_LSTM_model(p_input_shape, p_output_shape):

    input_layer = tfkl.Input(shape=p_input_shape, name='Input')

    lstm1 = tfkl.LSTM(64, activation='relu', return_sequences=True, input_shape=(p_input_shape))(input_layer)
    lstm2 = tfkl.LSTM(64, activation='relu')(lstm1)
    dense1 = tfkl.Dense(128)(lstm2)
    dropout = tfkl.Dropout(.3)(dense1)
    dense2 = tfkl.Dense(p_output_shape[-1]*p_output_shape[-2], activation='relu')(dropout)
    output_layer = tfkl.Reshape((p_output_shape[-2],p_output_shape[-1]))(dense2)
    output_layer = tfkl.Conv1D(p_output_shape[-1], 1, padding='same')(output_layer)

  # Connect input and output through the Model class
  model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

  # Compile the model
  model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=[tf.keras.metrics.RootMeanSquaredError()])

  # Return the model
  return model

In [ ]:
model = build_stacked_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [ ]:
# Train the model
callbacks = create_folders_and_callbacks(model_name='StackedLSTM')

history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=.1,
    callbacks=callbacks
).history

In [ ]:
model.save('StackedLSTM')

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17, 4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17, 4))
plt.plot(history['root_mean_squared_error'], label='Training RMSE', alpha=.8, color='#ff7f0e')
plt.plot(history['val_root_mean_squared_error'], label='Validation RMSE', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('RMSE')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18, 3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()